In [1]:
import cv2
import numpy as np
import random
from scipy.linalg import eigh
from scipy.spatial import distance
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
%matplotlib qt


# FUNCTIONS

In [2]:
def read_img(path):
    directory = os.path.abspath(path)
    img_names = np.array(os.listdir(directory))
    M = len(img_names)
    X, Y = 100, 100

    images = np.zeros((X*Y,M))

    for i,img in zip(range(M),img_names):
        temp = mpimg.imread(path + img)
        temp = cv2.resize(temp,(X,Y),cv2.INTER_AREA)
        if len(temp.shape) > 2:
            temp = np.dot(temp[...,:3], [0.299, 0.587, 0.144])   
        images[:,i] = temp.flatten()

    return images, img_names

def face_rec(images,k):
    N = images.shape[0]
    M = images.shape[1]
    A = np.zeros((N,M))
    
    mean_face = np.sum(images,axis=1)/M
    
    for i in range(M):
        A[:,i] = images[:,i] - mean_face
    
    lam, v = np.linalg.eig(np.cov(train.T))
    idx = np.argsort(lam)
    idx = idx[len(idx) - k:len(idx)]
    v = v[:,idx]
    
    u = A @ v
    
    return A, u, mean_face

def weights(A,u):
    if len(A.shape) > 1:
        N = A.shape[0]
        train_len = A.shape[1]
        w, proj = np.zeros((k,train_len)), np.zeros((N,train_len))
        for i in range(train_len):
            w[:,i] = u.T @ A[:,i] 
            proj[:,i] = (u @ w[:,i])
        
        return w, proj
    
    else:
        w = u.T @ A 
        proj = (u @ w)
        return w, proj

## Detection

In [3]:
path = '/home/karthick/PRML/Face-Detection/wash/Detected/'
directory = os.path.abspath(path) 
img_names = np.array(os.listdir(directory))
M = len(img_names)


cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

figure_size = 9

for i,img in zip(range(M),img_names):
    im = mpimg.imread(path + img)

    faces = faceCascade.detectMultiScale(im, scaleFactor=1.2, minNeighbors=5)
    
    for (x, y, w, h) in faces: 
        roi_color = im[y:y + h, x:x + w]
        cv2.imwrite('/home/karthick/PRML/Face-Detection/wash/test/' + img, roi_color)

FileNotFoundError: [Errno 2] No such file or directory: '/home/karthick/PRML/Face-Detection/wash/Detected'

# Main

In [21]:
path1 = '/home/karthick/PRML/Face-Detection/wash/train/'
path2 = '/home/karthick/PRML/Face-Detection/wash/Test/'

train, train_names = read_img(path1)
test, test_names = read_img(path2)

k = 15 # No. of principal components

A, u, mean_face = face_rec(train,k)

correct = 0
for i in range(len(test_names)):
    test_img = test[:,i]
    test_mean = test_img - mean_face

    train_w, train_eig = weights(A,u)
    test_w, test_eig = weights(test_mean,u)

    err = np.zeros((train_w.shape[1]))
    for j in range(train_w.shape[1]):
        err[j] = distance.euclidean(train_w[:,j],test_w)
    
    true = test_names[i][0:2]
    pred = train_names[np.where(err == min(err))[0][0]][0:2]
    
    if pred == true:
        correct = correct + 1 
        
    #print(true, pred)
    
print((correct/len(test_names)) * 100)

4.166666666666666
